In [1]:
# -aaa-
# f---b
# f---b
# f---b
# -ggg-
# e---c
# e---c
# e---c
# -ddd-

# 'abcdefg'
# '0123456'

In [2]:
from dataclasses import dataclass
from itertools import permutations
import pandas as pd

In [3]:
@dataclass
class Digit:
    value: int
    segments: []

In [4]:
with open('input.txt') as f:
    data = [line.strip('\n').split(' | ') for line in f]
    data = [(i.split(), o.split()) for i, o in data]

In [5]:
digs = [
    #number, segment positions
    (1, [1, 2]),
    (7, [0, 1, 2]),
    (4, [1, 2, 5, 6]),

    (2, [0, 1, 3, 4, 6]),
    (3, [0, 1, 2, 3, 6]),
    (5, [0, 2, 3, 5, 6]),
    (6, [0, 2, 3, 4, 5, 6]),
    (9, [0, 1, 2, 3, 5, 6]),
    (0, [0, 1, 2, 3, 4, 5]),

    (8, [0, 1, 2, 3, 4, 5, 6])
]

digits = {}

for item in digs:
    digits[item[0]] = Digit(value=item[0], segments=item[1])

In [6]:
digits

{1: Digit(value=1, segments=[1, 2]),
 7: Digit(value=7, segments=[0, 1, 2]),
 4: Digit(value=4, segments=[1, 2, 5, 6]),
 2: Digit(value=2, segments=[0, 1, 3, 4, 6]),
 3: Digit(value=3, segments=[0, 1, 2, 3, 6]),
 5: Digit(value=5, segments=[0, 2, 3, 5, 6]),
 6: Digit(value=6, segments=[0, 2, 3, 4, 5, 6]),
 9: Digit(value=9, segments=[0, 1, 2, 3, 5, 6]),
 0: Digit(value=0, segments=[0, 1, 2, 3, 4, 5]),
 8: Digit(value=8, segments=[0, 1, 2, 3, 4, 5, 6])}

In [7]:
def get_key(setups, pairs):

    for number, phrases in pairs:
        temp = pd.DataFrame()
        
        for phrase in phrases:
            subtemp = pd.DataFrame()
            cases = [''.join(p) for p in permutations(phrase)]

            for case in cases:
                if len(case) == 2:
                    subtemp = pd.concat([
                        subtemp,
                        setups[(setups[number[0]] == case[0]) & (setups[number[1]] == case[1])]
                        ])
                elif len(case) == 3:
                    subtemp = pd.concat([
                        subtemp,
                        setups[(setups[number[0]] == case[0]) & (setups[number[1]] == case[1]) & (setups[number[2]] == case[2])]
                        ])
                elif len(case) == 4:
                    subtemp = pd.concat([
                        subtemp,
                        setups[(setups[number[0]] == case[0]) & (setups[number[1]] == case[1]) & (setups[number[2]] == case[2]) & (setups[number[3]] == case[3])]
                        ])
                elif len(case) == 5:
                    subtemp = pd.concat([
                        subtemp,
                        setups[(setups[number[0]] == case[0]) & (setups[number[1]] == case[1]) & (setups[number[2]] == case[2]) & (setups[number[3]] == case[3]) & (setups[number[4]] == case[4])]
                        ])

            temp = pd.concat([temp, subtemp])

        setups = temp.copy()
        len(setups)

    return setups.iloc[0].str.cat()


In [8]:
#extracting segments based on key. 
def get_number(phrase, key):
    numbers = {
        0: key[0] + key[1] + key[2] + key[3] + key[4] + key[5],
        1: key[1] + key[2],
        2: key[0] + key[1] + key[3] + key[4] + key[6],
        3: key[0] + key[1] + key[2] + key[3] + key[6],
        4: key[1] + key[2] + key[5] + key[6],
        5: key[0] + key[2] + key[3] + key[5] + key[6],
        6: key[0] + key[2] + key[3] + key[4] + key[5] + key[6],
        7: key[0] + key[1] + key[2],
        8: key[0] + key[1] + key[2] + key[3] + key[4] + key[5] + key[6],
        9: key[0] + key[1] + key[2] + key[3] + key[5] + key[6],
    }

    for k, v in numbers.items():
        if ''.join(sorted(v)) == ''.join(sorted(phrase)):
            return k

In [9]:
def calculate_row(data, key):
    row_num = ''

    for item in data:
        row_num += str(get_number(item, key))

    return int(row_num)

In [10]:
def get_input_pack(phrase):

    #sorting by phrase length
    phrase = sorted([''.join(sorted(x)) for x in phrase], key=len)
    segs = {i: [] for i in range(2, 8)}

    for item in phrase:
        segs[len(item)].append(item)

    #completing number position and proper phrase (number 1 with phrase 'ab') 
    result = [
        (digits[1].segments, segs[2]),
        (digits[7].segments, segs[3]),
        (digits[4].segments, segs[4]),
        (digits[3].segments, segs[5]),
        (digits[2].segments, segs[5]),
        (digits[5].segments, segs[5])#,
        #(digits[6].segments, segs[6]),
        #(digits[9].segments, segs[6]),
        #(digits[0].segments, segs[6]),
        #(digits[8].segments, segs[7])
    ]

    return result

In [11]:
setups = pd.DataFrame([p for p in permutations('abcdefg')])
setups

,0,1,2,3,4,5,6
0,a,b,c,d,e,f,g
1,a,b,c,d,e,g,f
2,a,b,c,d,f,e,g
3,a,b,c,d,f,g,e
4,a,b,c,d,g,e,f
...,...,...,...,...,...,...,...
5035,g,f,e,d,a,c,b
5036,g,f,e,d,b,a,c
5037,g,f,e,d,b,c,a
5038,g,f,e,d,c,a,b


In [12]:
total = 0
round = 0

for input, output in data:
    round += 1
    sub_suma = ''

    #prepare (positions, segments) pairs
    input_package = get_input_pack(input)
    
    #extract current key
    key = get_key(setups, input_package)

    #extract current output value and add to total
    total += calculate_row(output, key)

    print(f'{round}. {total=}')

1. total=7861
2. total=12494
3. total=20384
4. total=25278
5. total=26493
6. total=31245
7. total=38853
8. total=48740
9. total=53627
10. total=54816
11. total=61934
12. total=65462
13. total=72592
14. total=75703
15. total=83811
16. total=91882
17. total=100654
18. total=104925
19. total=108483
20. total=116644
21. total=121400
22. total=130585
23. total=138465
24. total=143443
25. total=144919
26. total=153348
27. total=162923
28. total=171409
29. total=179544
30. total=186466
31. total=193249
32. total=197990
33. total=202364
34. total=206843
35. total=211617
36. total=211728
37. total=213312
38. total=222395
39. total=226802
40. total=233276
41. total=240907
42. total=242914
43. total=251038
44. total=258856
45. total=261960
46. total=267930
47. total=272301
48. total=273779
49. total=282567
50. total=284078
51. total=286148
52. total=294922
53. total=301905
54. total=303084
55. total=311929
56. total=316300
57. total=319953
58. total=325258
59. total=329445
60. total=332076
61. to